# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: None]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Data

In [4]:
with open('/content/drive/MyDrive/Dataset/energy data/county_id_to_name_map.json') as f:
    data = json.load(f)
    data={'county_id':[i for i in data.keys()],'County':[i for i in data.values()]}
county_map = pd.DataFrame(data)

In [5]:
weather_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/forecast_weather.csv')
weather_county_map=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/weather_station_to_county_mapping.csv')

In [6]:
print(weather_df.info(show_counts=True))
print(county_map.info(show_counts=True))
print(weather_county_map.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424512 entries, 0 to 3424511
Data columns (total 18 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   latitude                           3424512 non-null  float64
 1   longitude                          3424512 non-null  float64
 2   origin_datetime                    3424512 non-null  object 
 3   hours_ahead                        3424512 non-null  int64  
 4   temperature                        3424512 non-null  float64
 5   dewpoint                           3424512 non-null  float64
 6   cloudcover_high                    3424512 non-null  float64
 7   cloudcover_low                     3424512 non-null  float64
 8   cloudcover_mid                     3424512 non-null  float64
 9   cloudcover_total                   3424512 non-null  float64
 10  10_metre_u_wind_component          3424512 non-null  float64
 11  10_metre_v_wind_componen

In [7]:
weather_county_map['county'].unique()

array([nan, 10.,  1.,  7.,  6.,  9.,  0., 14.,  3., 13., 11.,  4.,  5.,
       15.,  8.,  2.])

# Creating Mapping for Counties

In [8]:
weather=weather_county_map[weather_county_map['county_name'].notnull()].sort_values(['latitude','longitude'])[['county','county_name']].drop_duplicates()
county_mapp=weather.set_index('county').to_dict()['county_name']

# Finding the best Hyperparameters for knn

In [9]:
weather=weather_county_map[weather_county_map['county_name'].notnull()].sort_values(['latitude','longitude'])

X=np.array(weather[['latitude', 'longitude']])
y=np.array(weather[['county']])
y=y.reshape((y.shape[0],))

k=15
knn_classifier = KNeighborsClassifier(n_neighbors=k)

param_grid = {'p': [1, 2, 3],'weights':['uniform', 'distance'],'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],'leaf_size':[25,30]}
grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X, y)

print("Best hyperparameters:", grid_search.best_params_)
best_knn_model = grid_search.best_estimator_
y_pred = best_knn_model.predict(X)

accuracy = accuracy_score(y, y_pred)
print(f"Accuracy with best hyperparameters: {accuracy}")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best hyperparameters: {'algorithm': 'auto', 'leaf_size': 25, 'p': 2, 'weights': 'distance'}
Accuracy with best hyperparameters: 1.0


#Training knn model for county Map with best hyperparameters

In [10]:
knn_classifier = KNeighborsClassifier(n_neighbors=k, algorithm='auto',leaf_size=25,p=2,weights='distance')
knn_classifier.fit(X, y)
y_pred = knn_classifier.predict(X)

accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy}")

report = classification_report(y, y_pred)
print("Classification Report:\n", report)

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         6
         1.0       1.00      1.00      1.00         1
         2.0       1.00      1.00      1.00         4
         3.0       1.00      1.00      1.00         3
         4.0       1.00      1.00      1.00         3
         5.0       1.00      1.00      1.00         4
         6.0       1.00      1.00      1.00         2
         7.0       1.00      1.00      1.00         5
         8.0       1.00      1.00      1.00         1
         9.0       1.00      1.00      1.00         3
        10.0       1.00      1.00      1.00         4
        11.0       1.00      1.00      1.00         4
        13.0       1.00      1.00      1.00         1
        14.0       1.00      1.00      1.00         3
        15.0       1.00      1.00      1.00         5

    accuracy                           1.00        49
   macro avg       1.00      1.00      1.0

# Applying knn model to predict rest of the unknown coordinates

In [11]:
weather=weather_county_map[weather_county_map['county_name'].isnull()].sort_values(['latitude','longitude']).drop_duplicates(subset=['latitude','longitude']).reset_index(drop=True).drop(['county','county_name'],axis=1)
X=np.array(weather[['latitude','longitude']])
county_predict=knn_classifier.predict(X)
weather=pd.concat([weather, pd.DataFrame(county_predict, columns=['county'])],axis=1)
weather_county_map=weather_county_map.merge(weather,on=['latitude','longitude'], how='left', suffixes=('','_right_'))
weather_county_map['county']=np.where(weather_county_map['county'].isnull(),weather_county_map['county_right_'],weather_county_map['county'])
weather_county_map.drop('county_right_', axis=1, inplace=True)
weather_county_map['county_name']=weather_county_map['county']
weather_county_map['county_name']=weather_county_map['county_name'].map(county_mapp)
weather_county_map[['county_name','latitude',	'longitude',	'county']]
weather_county_map

,county_name,longitude,latitude,county
0,Saaremaa,21.7,57.6,10.0
1,Saaremaa,21.7,57.9,10.0
2,Saaremaa,21.7,58.2,10.0
3,Saaremaa,21.7,58.5,10.0
4,Saaremaa,21.7,58.8,10.0
5,Saaremaa,21.7,59.1,10.0
6,Saaremaa,21.7,59.4,10.0
7,Saaremaa,21.7,59.7,10.0
8,Saaremaa,22.2,57.6,10.0
9,Saaremaa,22.2,57.9,10.0


# Retraining knn model with the new found values

In [ ]:
X=np.array(weather_county_map[['latitude', 'longitude']])
y=np.array(weather_county_map[['county']])
y=y.reshape((y.shape[0],))

k=15
knn_classifier = KNeighborsClassifier(n_neighbors=k)

param_grid = {'p': [1, 2, 3],'weights':['uniform', 'distance'],'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],'leaf_size':[25,30]}
grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X, y)

print("Best hyperparameters:", grid_search.best_params_)
best_knn_model = grid_search.best_estimator_
y_pred = best_knn_model.predict(X)

accuracy = accuracy_score(y, y_pred)
print(f"Accuracy with best hyperparameters: {accuracy}")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best hyperparameters: {'algorithm': 'auto', 'leaf_size': 25, 'p': 1, 'weights': 'distance'}
Accuracy with best hyperparameters: 1.0


In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=k, algorithm='auto',leaf_size=25,p=1,weights='distance')
knn_classifier.fit(X, y)
y_pred = knn_classifier.predict(X)

accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy}")

report = classification_report(y, y_pred)
print("Classification Report:\n", report)

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        12
         1.0       1.00      1.00      1.00         1
         2.0       1.00      1.00      1.00        12
         3.0       1.00      1.00      1.00         3
         4.0       1.00      1.00      1.00         3
         5.0       1.00      1.00      1.00         6
         6.0       1.00      1.00      1.00         2
         7.0       1.00      1.00      1.00        15
         8.0       1.00      1.00      1.00         1
         9.0       1.00      1.00      1.00         3
        10.0       1.00      1.00      1.00        31
        11.0       1.00      1.00      1.00         4
        13.0       1.00      1.00      1.00         1
        14.0       1.00      1.00      1.00         5
        15.0       1.00      1.00      1.00        13

    accuracy                           1.00       112
   macro avg       1.00      1.00      1.0

# Applying on weather df

In [ ]:
weather=np.array(weather_df[['latitude', 'longitude']])
county_predict=knn_classifier.predict(weather)
weather = np.concatenate((weather, county_predict.reshape(county_predict.shape[0], 1)), axis=1)
weather=pd.DataFrame(weather, columns=['latitudee','longitudee','county'])
weather['county_name']=weather['county']
weather['county_name']=weather['county_name'].map(county_mapp)
weather_df=pd.concat([weather_df,weather],axis=1)
weather_df.drop(['latitudee','longitudee'],axis=1,inplace=True)

In [ ]:
X=[]
county_predict=[]
weather=[]
weather_df.to_csv('/content/drive/MyDrive/Dataset/energy data/forecast_weather_updated.csv', index=False)
weather_df=[]

# Applying on historical weather df

In [ ]:
hist_weather_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/historical_weather.csv')
print(hist_weather_df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710802 entries, 0 to 1710801
Data columns (total 18 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   datetime                1710802 non-null  object 
 1   temperature             1710802 non-null  float64
 2   dewpoint                1710802 non-null  float64
 3   rain                    1710802 non-null  float64
 4   snowfall                1710802 non-null  float64
 5   surface_pressure        1710802 non-null  float64
 6   cloudcover_total        1710802 non-null  int64  
 7   cloudcover_low          1710802 non-null  int64  
 8   cloudcover_mid          1710802 non-null  int64  
 9   cloudcover_high         1710802 non-null  int64  
 10  windspeed_10m           1710802 non-null  float64
 11  winddirection_10m       1710802 non-null  int64  
 12  shortwave_radiation     1710802 non-null  float64
 13  direct_solar_radiation  1710802 non-null  float64
 14  di

In [ ]:
weather=np.array(hist_weather_df[['latitude', 'longitude']])
county_predict=knn_classifier.predict(weather)
weather = np.concatenate((weather, county_predict.reshape(county_predict.shape[0], 1)), axis=1)
weather=pd.DataFrame(weather, columns=['latitudee','longitudee','county'])
weather['county_name']=weather['county']
weather['county_name']=weather['county_name'].map(county_mapp)
hist_weather_df=pd.concat([hist_weather_df,weather],axis=1)
hist_weather_df.drop(['latitudee','longitudee'],axis=1,inplace=True)

In [ ]:
X=[]
county_predict=[]
weather=[]
hist_weather_df.to_csv('/content/drive/MyDrive/Dataset/energy data/historical_weather_updated.csv', index=False)
hist_weather_df=[]

# Next

In [ ]:
gas_price_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/gas_prices.csv')
train_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/train.csv')
client_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/client.csv')
e_price_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/electricity_prices.csv')

In [ ]:
print(gas_price_df.info(show_counts=True))
print(train_df.info(show_counts=True))
print(client_df.info(show_counts=True))
print(e_price_df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   forecast_date          637 non-null    object 
 1   lowest_price_per_mwh   637 non-null    float64
 2   highest_price_per_mwh  637 non-null    float64
 3   origin_date            637 non-null    object 
 4   data_block_id          637 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 25.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   county              2018352 non-null  int64  
 1   is_business         2018352 non-null  int64  
 2   product_type        2018352 non-null  int64  
 3   target              2017824 non-null  float64
 4   is_consumption      2018352 non-null  int

In [ ]:
train_df=train_df.merge(client_df,on=['product_type','is_business','county','data_block_id'],how='left')
train_df.dropna(subset=['target'],inplace=True)
gas_price_df['data_block_id']=gas_price_df['data_block_id']-1
e_price_df['data_block_id']=e_price_df['data_block_id']-1
e_price_df['forecast_date']=pd.to_datetime(e_price_df['forecast_date'])
e_price_df['forecast_time']=e_price_df['forecast_date'].dt.time
e_price_df['forecast_date']=e_price_df['forecast_date'].dt.date
train_df=train_df.merge(gas_price_df, on='data_block_id',suffixes=['','_gas_'],how='left')
gas_price_df=[]
client_df=[]

In [ ]:
e_price_df.head(100)

,forecast_date,euros_per_mwh,origin_date,data_block_id,forecast_time
0,2021-09-01,92.51,2021-08-31 00:00:00,0,00:00:00
1,2021-09-01,88.90,2021-08-31 01:00:00,0,01:00:00
2,2021-09-01,87.35,2021-08-31 02:00:00,0,02:00:00
3,2021-09-01,86.88,2021-08-31 03:00:00,0,03:00:00
4,2021-09-01,88.43,2021-08-31 04:00:00,0,04:00:00
5,2021-09-01,93.58,2021-08-31 05:00:00,0,05:00:00
6,2021-09-01,118.70,2021-08-31 06:00:00,0,06:00:00
7,2021-09-01,135.44,2021-08-31 07:00:00,0,07:00:00
8,2021-09-01,140.72,2021-08-31 08:00:00,0,08:00:00
9,2021-09-01,130.00,2021-08-31 09:00:00,0,09:00:00


In [ ]:
train_df['datetime']=pd.to_datetime(train_df['datetime'])
train_df['date_target']=train_df['datetime'].dt.date
train_df['time_target']=train_df['datetime'].dt.time
train_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2017824 entries, 0 to 2017823
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   county                 2017824 non-null  int64         
 1   is_business            2017824 non-null  int64         
 2   product_type           2017824 non-null  int64         
 3   target                 2017824 non-null  float64       
 4   is_consumption         2017824 non-null  int64         
 5   datetime               2017824 non-null  datetime64[ns]
 6   data_block_id          2017824 non-null  int64         
 7   row_id                 2017824 non-null  int64         
 8   prediction_unit_id     2017824 non-null  int64         
 9   eic_count              2009184 non-null  float64       
 10  installed_capacity     2009184 non-null  float64       
 11  date                   2009184 non-null  object        
 12  forecast_date          20147

In [ ]:
weather_forecast_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/forecast_weather_updated.csv')
hist_weather_df=pd.read_csv('/content/drive/MyDrive/Dataset/energy data/historical_weather_updated.csv')
del weather_df

In [ ]:
print(weather_forecast_df.shape)
weather_forecast_df.info(show_counts=True)
weather_forecast_df.describe()
weather_forecast_df=weather_forecast_df.groupby(['origin_datetime','data_block_id','forecast_datetime','county','county_name'],as_index=False)['hours_ahead','temperature','dewpoint','cloudcover_high','cloudcover_low','cloudcover_mid','cloudcover_total',
                                                                                                                                                       '10_metre_u_wind_component','10_metre_v_wind_component','direct_solar_radiation','surface_solar_radiation_downwards',
                                                                                                                                                       'snowfall','total_precipitation'].mean()
weather_forecast_df['forecast_datetime']=pd.to_datetime(weather_forecast_df['forecast_datetime'])
weather_forecast_df['date']=weather_forecast_df['forecast_datetime'].dt.date
weather_forecast_df['time']=weather_forecast_df['forecast_datetime'].dt.time
weather_forecast_df.drop(['forecast_datetime'],axis=1,inplace=True)

(3424512, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424512 entries, 0 to 3424511
Data columns (total 20 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   latitude                           3424512 non-null  float64
 1   longitude                          3424512 non-null  float64
 2   origin_datetime                    3424512 non-null  object 
 3   hours_ahead                        3424512 non-null  int64  
 4   temperature                        3424512 non-null  float64
 5   dewpoint                           3424512 non-null  float64
 6   cloudcover_high                    3424512 non-null  float64
 7   cloudcover_low                     3424512 non-null  float64
 8   cloudcover_mid                     3424512 non-null  float64
 9   cloudcover_total                   3424512 non-null  float64
 10  10_metre_u_wind_component          3424512 non-null  float64
 11  10_metre_v

<ipython-input-13-de07d7a4de6e>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weather_forecast_df=weather_forecast_df.groupby(['origin_datetime','data_block_id','forecast_datetime','county','county_name'],as_index=False)['hours_ahead','temperature','dewpoint','cloudcover_high','cloudcover_low','cloudcover_mid','cloudcover_total',


In [ ]:
print(hist_weather_df.shape)
hist_weather_df.info(show_counts=True)
hist_weather_df.describe()
hist_weather_df=hist_weather_df.groupby(['datetime','data_block_id','county','county_name'],as_index=False)['rain','temperature','dewpoint','surface_pressure','cloudcover_high','cloudcover_low','cloudcover_mid','cloudcover_total',
                                                                                                                                                       'windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation',
                                                                                                                                                       'snowfall'].mean()
hist_weather_df['datetime']=pd.to_datetime(hist_weather_df['datetime'])
hist_weather_df['date']=hist_weather_df['datetime'].dt.date
hist_weather_df['time']=hist_weather_df['datetime'].dt.time
hist_weather_df.drop(['datetime'],axis=1,inplace=True)
hist_weather_df.shape

(1710802, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710802 entries, 0 to 1710801
Data columns (total 20 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   datetime                1710802 non-null  object 
 1   temperature             1710802 non-null  float64
 2   dewpoint                1710802 non-null  float64
 3   rain                    1710802 non-null  float64
 4   snowfall                1710802 non-null  float64
 5   surface_pressure        1710802 non-null  float64
 6   cloudcover_total        1710802 non-null  int64  
 7   cloudcover_low          1710802 non-null  int64  
 8   cloudcover_mid          1710802 non-null  int64  
 9   cloudcover_high         1710802 non-null  int64  
 10  windspeed_10m           1710802 non-null  float64
 11  winddirection_10m       1710802 non-null  int64  
 12  shortwave_radiation     1710802 non-null  float64
 13  direct_solar_radiation  1710802 non-null  f

<ipython-input-14-1348ede2cc62>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hist_weather_df=hist_weather_df.groupby(['datetime','data_block_id','county','county_name'],as_index=False)['rain','temperature','dewpoint','surface_pressure','cloudcover_high','cloudcover_low','cloudcover_mid','cloudcover_total',


(229125, 19)

In [ ]:
train_df.dropna(subset=['eic_count','installed_capacity','date','forecast_date','lowest_price_per_mwh','highest_price_per_mwh','origin_date'],inplace=True)

In [ ]:
print(train_df.info(show_counts=True))
print(e_price_df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2006064 entries, 5856 to 2014703
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   county                 2006064 non-null  int64         
 1   is_business            2006064 non-null  int64         
 2   product_type           2006064 non-null  int64         
 3   target                 2006064 non-null  float64       
 4   is_consumption         2006064 non-null  int64         
 5   datetime               2006064 non-null  datetime64[ns]
 6   data_block_id          2006064 non-null  int64         
 7   row_id                 2006064 non-null  int64         
 8   prediction_unit_id     2006064 non-null  int64         
 9   eic_count              2006064 non-null  float64       
 10  installed_capacity     2006064 non-null  float64       
 11  date                   2006064 non-null  object        
 12  forecast_date          20

In [ ]:
scalerforecast=StandardScaler()
scalerhist=StandardScaler()
hist=['rain', 'temperature', 'dewpoint', 'surface_pressure','cloudcover_high', 'cloudcover_low', 'cloudcover_mid','cloudcover_total', 'windspeed_10m', 'winddirection_10m','shortwave_radiation', 'direct_solar_radiation', 'diffuse_radiation','snowfall']
forecast=['hours_ahead','temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low','cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component','10_metre_v_wind_component','direct_solar_radiation','surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
weather_forecast_df[forecast]=scalerforecast.fit_transform(weather_forecast_df[forecast])
hist_weather_df[hist]=scalerhist.fit_transform(hist_weather_df[hist])

In [ ]:
weather_forecast_df['data_block_id']=weather_forecast_df['data_block_id']+1
hist_weather_df['data_block_id']=hist_weather_df['data_block_id']-1

# TESTING HERE

In [ ]:
checking_n_unique=train_df[['county','is_business','product_type','is_consumption']].drop_duplicates()
empt = []
for index, row in checking_n_unique.iterrows():
  filtered_df = train_df[
      (train_df['county'] == row['county']) &  # Use bitwise AND (&) for filtering
      (train_df['is_business'] == row['is_business']) &
      (train_df['product_type'] == row['product_type']) &
      (train_df['is_consumption'] == row['is_consumption'])
  ]
  empt.append(filtered_df)
del train_df

In [ ]:
len(empt[0].shape)

2

# DONE TESTING

In [ ]:
timelist=np.sort(train_df['datetime'].unique())
print(timelist)

['2021-09-03T00:00:00.000000000' '2021-09-03T01:00:00.000000000'
 '2021-09-03T02:00:00.000000000' ... '2023-05-30T21:00:00.000000000'
 '2023-05-30T22:00:00.000000000' '2023-05-30T23:00:00.000000000']


In [ ]:
subset_train=empt[0]
subset_elec=e_price_df[e_price_df['data_block_id'].isin(list(subset_train['data_block_id'].unique()))]
subset_hist=hist_weather_df[hist_weather_df['data_block_id'].isin(list(subset_train['data_block_id'].unique()))]
subset_forecast=weather_forecast_df[weather_forecast_df['data_block_id'].isin(list(subset_train['data_block_id'].unique()))]
subset_train=subset_train.merge(e_price_df, left_on=['data_block_id','time_target'],right_on=['data_block_id','forecast_time'], how='left', suffixes=['_gas_','_e_'])
subset_forecast=subset_forecast.merge(subset_hist, left_on=['data_block_id','county','date','time'],right_on=['data_block_id','county','date','time'], how='outer',suffixes=['_wforecast_','_whist_'])
subset_train=subset_train.merge(subset_forecast, left_on=['data_block_id','county','date_target','time_target'],right_on=['data_block_id','county','date','time'], how='left',suffixes=['_train_',''])

In [ ]:
to_drop=['data_block_id','row_id','prediction_unit_id','county_name_whist_','county_name_wforecast_','datetime','forecast_date_gas_','forecast_date_e_','forecast_time','date','time']
subset_train.drop(to_drop,axis=1,inplace=True)
subset_train.rename(columns={'date_train_':'client_entry_date'},inplace=True)

In [ ]:
subset_train.select_dtypes(exclude=['int', 'float']).info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15236 entries, 0 to 15235
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_entry_date  15236 non-null  object
 1   origin_date_gas_   15236 non-null  object
 2   date_target        15236 non-null  object
 3   time_target        15236 non-null  object
 4   origin_date_e_     15234 non-null  object
 5   origin_datetime    1611 non-null   object
dtypes: object(6)
memory usage: 833.2+ KB


In [ ]:
subset_train['date_target']=pd.to_datetime(subset_train['date_target'].astype(str) + ' ' + subset_train['time_target'].astype(str))
subset_train.drop(['time_target'],inplace=True,axis=1)
for i in list(subset_train.select_dtypes(exclude=['int', 'float']).columns):
  subset_train[i]=pd.to_datetime(subset_train[i].astype(str))

subset_train['target_year']=subset_train['date_target'].dt.year
subset_train['target_month']=subset_train['date_target'].dt.month
subset_train['target_day']=subset_train['date_target'].dt.day
subset_train['target_hour']=subset_train['date_target'].dt.hour
subset_train['target_day_of_week']=subset_train['date_target'].dt.dayofweek

subset_train['forecast_year']=subset_train['origin_datetime'].dt.year
subset_train['forecast_month']=subset_train['origin_datetime'].dt.month
subset_train['forecast_day']=subset_train['origin_datetime'].dt.day
subset_train['forecast_hour']=subset_train['origin_datetime'].dt.hour

subset_train['e_price_year']=subset_train['origin_date_e_'].dt.year
subset_train['e_price_month']=subset_train['origin_date_e_'].dt.month
subset_train['e_price_day']=subset_train['origin_date_e_'].dt.day
subset_train['e_price_hour']=subset_train['origin_date_e_'].dt.hour
subset_train['e_price_day_of_week']=subset_train['origin_date_e_'].dt.dayofweek

subset_train['gas_price_year']=subset_train['origin_date_gas_'].dt.year
subset_train['gas_price_month']=subset_train['origin_date_gas_'].dt.month
subset_train['gas_price_day']=subset_train['origin_date_gas_'].dt.day
subset_train['gas_price_day_of_week']=subset_train['origin_date_gas_'].dt.dayofweek

subset_train.drop(['client_entry_date',"origin_date_gas_",	'date_target',	'origin_date_e_',	'origin_datetime'],axis=1,inplace=True)
subset_train.fillna(-1000,inplace=True)

In [ ]:
subset_train.sort_values(['target_year','target_month','target_day','target_hour'])

In [ ]:
checking=subset_train[['county','is_business','product_type','is_consumption']]
checking=checking.drop_duplicates(['county','is_business','product_type','is_consumption'])
checking

,county,is_business,product_type,is_consumption
0,0,0,1,0


In [ ]:
subset_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15236 entries, 0 to 15235
Data columns (total 55 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   county                             15236 non-null  int64  
 1   is_business                        15236 non-null  int64  
 2   product_type                       15236 non-null  int64  
 3   target                             15236 non-null  float64
 4   is_consumption                     15236 non-null  int64  
 5   eic_count                          15236 non-null  float64
 6   installed_capacity                 15236 non-null  float64
 7   lowest_price_per_mwh               15236 non-null  float64
 8   highest_price_per_mwh              15236 non-null  float64
 9   euros_per_mwh                      15236 non-null  float64
 10  hours_ahead                        15236 non-null  float64
 11  temperature_wforecast_             15236 non-null  flo

In [ ]:
len(empt[42])

14444

In [ ]:
# Function to create and merge datasets on runtime
def generate_and_merge_data(val):
    # Generate data for each table
    global empt
    global e_price_df
    global weather_forecast_df
    global hist_weather_df
    c=0
    if val==9:
      e=len(empt)
    else:
      e=(val+1)*16
    for i in range(val*16,e):
      subset_train=empt[i]
      subset_elec=e_price_df[e_price_df['data_block_id'].isin(list(subset_train['data_block_id'].unique()))]
      subset_hist=hist_weather_df[hist_weather_df['data_block_id'].isin(list(subset_train['data_block_id'].unique()))]
      subset_forecast=weather_forecast_df[weather_forecast_df['data_block_id'].isin(list(subset_train['data_block_id'].unique()))]
      subset_train=subset_train.merge(e_price_df, left_on=['data_block_id','time_target'],right_on=['data_block_id','forecast_time'], how='left', suffixes=['_gas_','_e_'])
      subset_forecast=subset_forecast.merge(subset_hist, left_on=['data_block_id','county','date','time'],right_on=['data_block_id','county','date','time'], how='outer',suffixes=['_wforecast_','_whist_'])
      subset_train=subset_train.merge(subset_forecast, left_on=['data_block_id','county','date_target','time_target'],right_on=['data_block_id','county','date','time'], how='left',suffixes=['_train_',''])

      to_drop=['data_block_id','row_id','prediction_unit_id','county_name_whist_','county_name_wforecast_','datetime','forecast_date_gas_','forecast_date_e_','forecast_time','date','time']
      subset_train.drop(to_drop,axis=1,inplace=True)
      subset_train.rename(columns={'date_train_':'client_entry_date'},inplace=True)

      subset_train['date_target']=pd.to_datetime(subset_train['date_target'].astype(str) + ' ' + subset_train['time_target'].astype(str))
      subset_train.drop(['time_target'],inplace=True,axis=1)
      for j in list(subset_train.select_dtypes(exclude=['int', 'float']).columns):
        subset_train[j]=pd.to_datetime(subset_train[j].astype(str))

      subset_train['target_year']=subset_train['date_target'].dt.year
      subset_train['target_month']=subset_train['date_target'].dt.month
      subset_train['target_day']=subset_train['date_target'].dt.day
      subset_train['target_hour']=subset_train['date_target'].dt.hour
      subset_train['target_day_of_week']=subset_train['date_target'].dt.dayofweek

      subset_train['forecast_year']=subset_train['origin_datetime'].dt.year
      subset_train['forecast_month']=subset_train['origin_datetime'].dt.month
      subset_train['forecast_day']=subset_train['origin_datetime'].dt.day
      subset_train['forecast_hour']=subset_train['origin_datetime'].dt.hour

      subset_train['e_price_year']=subset_train['origin_date_e_'].dt.year
      subset_train['e_price_month']=subset_train['origin_date_e_'].dt.month
      subset_train['e_price_day']=subset_train['origin_date_e_'].dt.day
      subset_train['e_price_hour']=subset_train['origin_date_e_'].dt.hour
      subset_train['e_price_day_of_week']=subset_train['origin_date_e_'].dt.dayofweek

      subset_train['gas_price_year']=subset_train['origin_date_gas_'].dt.year
      subset_train['gas_price_month']=subset_train['origin_date_gas_'].dt.month
      subset_train['gas_price_day']=subset_train['origin_date_gas_'].dt.day
      subset_train['gas_price_day_of_week']=subset_train['origin_date_gas_'].dt.dayofweek

      subset_train.drop(['client_entry_date',"origin_date_gas_",	'date_target',	'origin_date_e_',	'origin_datetime'],axis=1,inplace=True)
      subset_train.fillna(-1000,inplace=True)

      # Prepare input features and target variable
      X_sub = np.array(subset_train.drop('target',axis=1))
      y_sub = np.array(subset_train[['target']])  # Adjust with your target variable name
      pad_width = ((0, 15236-X_sub.shape[0]),(0,0))  # Calculate the difference in time steps (15236 - 14444)
      X_sub = np.pad(X_sub, pad_width=pad_width, mode='constant', constant_values=-1)
      y_sub = np.pad(y_sub, pad_width=pad_width, mode='constant', constant_values=-1)
      X_sub = np.reshape(X_sub, (1, X_sub.shape[0], X_sub.shape[1]))
      y_sub = np.reshape(y_sub, (1, y_sub.shape[0], y_sub.shape[1]))  # Reshape to 3D
      if c==0:
        X=X_sub
        y=y_sub
        c=1
      else:
        X = np.append(X, X_sub, axis=0)
        y = np.append(y, y_sub, axis=0)
    return X,y

In [ ]:
import tensorflow as tf
class SaveWeightsCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    model.save_weights("/content/drive/MyDrive/Dataset/energy data/model_weights.h5")  # Replace with your save path

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Dropout, Reshape
# Define your model architecture here (replace with your code)
model = Sequential([
    Conv1D(filters=48, kernel_size=1, dilation_rate=2, strides=1,activation='relu',input_shape=[15236,54]),
    Conv1D(filters=40, kernel_size=1, dilation_rate=2, strides=1,activation='relu'),
    #Reshape((15236, 64), input_shape=(None, 15236, 64)),
    LSTM(512, return_sequences=True),
    LSTM(256, return_sequences=True),
    Dropout(0.3),
    Conv1D(filters=32, kernel_size=1, dilation_rate=2, strides=1,activation='relu'),
    LSTM(128, return_sequences=True),
    Dense(64,activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model once (outside the loop)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])  # Adjust optimizer, loss, and metrics as needed


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 15236, 48)         2640      
                                                                 
 conv1d_1 (Conv1D)           (None, 15236, 40)         1960      
                                                                 
 lstm (LSTM)                 (None, 15236, 512)        1132544   
                                                                 
 lstm_1 (LSTM)               (None, 15236, 256)        787456    
                                                                 
 dropout (Dropout)           (None, 15236, 256)        0         
                                                                 
 conv1d_2 (Conv1D)           (None, 15236, 32)         8224      
                                                                 
 lstm_2 (LSTM)               (None, 15236, 128)        8

In [ ]:
for loop_index in range(10):
  X_train,y_train=generate_and_merge_data(loop_index)
  # Load weights from the previous iteration (or initial weights for the first loop)
  if loop_index > 0:
    weight_path = f"/content/drive/MyDrive/weights_{loop_index-1}.h5"  # Adjust path format
    model.load_weights(weight_path)

  # Train the model by continuing from the loaded weights
  model.fit(X_train, y_train, epochs=30)  # Adjust epochs_per_loop

  # Save weights for this iteration
  save_path = f"/content/drive/MyDrive/weights_{loop_index}.h5"  # Adjust path format
  model.save_weights(save_path)

Epoch 1/30
1/1 [==============================] - 14s 14s/step - loss: 5032345.5000 - accuracy: 0.1464
Epoch 2/30
1/1 [==============================] - 3s 3s/step - loss: 5031766.5000 - accuracy: 0.1464
Epoch 3/30
1/1 [==============================] - 3s 3s/step - loss: 5030868.5000 - accuracy: 3.4868e-04
Epoch 4/30
1/1 [==============================] - 3s 3s/step - loss: 5029891.0000 - accuracy: 3.1176e-04
Epoch 5/30
1/1 [==============================] - 3s 3s/step - loss: 5028955.0000 - accuracy: 2.8715e-04
Epoch 6/30
1/1 [==============================] - 3s 3s/step - loss: 5028052.5000 - accuracy: 2.7484e-04
Epoch 7/30
1/1 [==============================] - 3s 3s/step - loss: 5027171.5000 - accuracy: 2.6254e-04
Epoch 8/30
1/1 [==============================] - 3s 3s/step - loss: 5026275.0000 - accuracy: 2.6254e-04
Epoch 9/30
1/1 [==============================] - 3s 3s/step - loss: 5025305.0000 - accuracy: 2.6254e-04
Epoch 10/30
1/1 [==============================] - 3s 3s/step

IndexError: list index out of range

In [ ]:
import tensorflow as tf


time_steps=train_df['datetime'].nunique()
model = Sequential()
sequence_length=1
num_features=54
model.add(LSTM(50, activation='relu', input_shape=(None, num_features)))
model.add(Dense(1))  # Assuming a single output for regression
validation_interval = 10

model.compile(optimizer='adam', loss='mse')

accumulated_state = None

# Example of how to use the function in your training loop
for timestep, val in enumerate(timelist):
    # Generate and merge data on runtime
    X_train, y_train = generate_and_merge_data(timestep, val)
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    # Reshape X_train to match the input shape of the LSTM model
    X_train = X_train.reshape((X_train.shape[0], 1, num_features))

    # Train the model with a rolling window of past timesteps
    if timestep >= sequence_length:
        if accumulated_state is None:
            # If this is the first time, initialize the accumulated_state
            accumulated_state = model.get_weights()

        model.set_weights(accumulated_state)
        model.train_on_batch(X_train, y_train)
        accumulated_state = model.get_weights()
    else:
      model.train_on_batch(X_train, y_train)
      accumulated_state = model.get_weights()

    if timestep % validation_interval == 0:
        X_val, y_val = generate_and_merge_data(timestep, val)
        X_val=np.array(X_val)
        y_val=np.array(y_val)
        X_val = X_val.reshape((X_val.shape[0], 1, num_features))
        loss = model.evaluate(X_val, y_val, verbose=0)
        print(f'Timestep: {timestep}, Validation Loss: {loss}')

Timestep: 0, Validation Loss: 408037.5
Timestep: 10, Validation Loss: 625017.1875
Timestep: 20, Validation Loss: 529190.1875
Timestep: 30, Validation Loss: 242509.796875
Timestep: 40, Validation Loss: 186391.6875
Timestep: 50, Validation Loss: 227905.765625
Timestep: 60, Validation Loss: 234345.75
Timestep: 70, Validation Loss: 281064.5
Timestep: 80, Validation Loss: 603515.4375
Timestep: 90, Validation Loss: 513070.375
Timestep: 100, Validation Loss: 241906.453125
Timestep: 110, Validation Loss: 577859.9375
Timestep: 120, Validation Loss: 277255.40625
Timestep: 130, Validation Loss: 664834.25
Timestep: 140, Validation Loss: 426968.4375
Timestep: 150, Validation Loss: 349271.71875
Timestep: 160, Validation Loss: 349309.34375
Timestep: 170, Validation Loss: 247732.84375
Timestep: 180, Validation Loss: 558871.125
Timestep: 190, Validation Loss: 300288.96875
Timestep: 200, Validation Loss: 214215.9375
Timestep: 210, Validation Loss: 176279.734375
Timestep: 220, Validation Loss: 216031.453

# Check Later


In [ ]:
train_df.shape
e_price_df.sort_values('data_block_id').head(100)
client_df['county'].value_counts(dropna=False)

0     4422
11    4115
7     3596
5     3149
15    3089
4     3058
10    2797
14    2611
3     2544
9     2544
13    2514
2     2392
1     1908
8     1908
6      636
12     636
Name: county, dtype: int64

In [ ]:
train_df=train_df.merge(e_price_df, on='data_block_id',how='left',suffixes=('','e_price'))
#train_df=train_df.merge(gas_price_df, on='data_block_id',how='left',suffixes=('','gas_price'))
train_df.head(100)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,euros_per_mwh,origin_date
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,NaN,NaN,NaN
5,0,0,3,656.859,1,2021-09-01 00:00:00,0,5,2,NaN,NaN,NaN
6,0,1,0,0.000,0,2021-09-01 00:00:00,0,6,3,NaN,NaN,NaN
7,0,1,0,59.000,1,2021-09-01 00:00:00,0,7,3,NaN,NaN,NaN
8,0,1,1,0.000,0,2021-09-01 00:00:00,0,8,4,NaN,NaN,NaN
9,0,1,1,501.760,1,2021-09-01 00:00:00,0,9,4,NaN,NaN,NaN


In [ ]:
train_df.info(null_counts=True)

<ipython-input-6-3d6dc96a2c30>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  train_df.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 48366672 entries, 0 to 48366671
Data columns (total 12 columns):
 #   Column              Non-Null Count     Dtype  
---  ------              --------------     -----  
 0   county              48366672 non-null  int64  
 1   is_business         48366672 non-null  int64  
 2   product_type        48366672 non-null  int64  
 3   target              48354000 non-null  float64
 4   is_consumption      48366672 non-null  int64  
 5   datetime            48366672 non-null  object 
 6   data_block_id       48366672 non-null  int64  
 7   row_id              48366672 non-null  int64  
 8   prediction_unit_id  48366672 non-null  int64  
 9   forecast_date       48363744 non-null  object 
 10  euros_per_mwh       48363744 non-null  float64
 11  origin_date         48363744 non-null  object 
dtypes: float64(2), int64(7), object(3)
memory usage: 4.7+ GB
